<a href="https://colab.research.google.com/github/Murcha1990/ML_AI24/blob/main/Hometasks/Base/ML_AI24_HT7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Домашнее задание 7: Fraud Detection Competition**

Добро пожаловать на финальное домашнее задание курса! Домашнее задание направлено на систематизацию всех знаний, полученных в процессе учебы.

В нём Вы потренируетесь применять навыки построения пайплайнов машинного обучения, приобретенные в курсе от точки разведочного анализа данных до построения и валидации моделей машинного обучения.

## **Задача**
**Вы будете решать задачу определения фрода:**

https://www.kaggle.com/competitions/fraud-detection-24

**Вам нужно будет:**
- в jupyter notebook провести исследование данных;
- в нём же построить модели и оценить их качество;
- отправить посылку на Kaggle.

Более подробное описание шагов - в ноутбуке ниже.

## **Оценивание и баллы**
- В EDA и во всей работе будут оцениваться полнота и **выводы**;
- При обучении моделей старайтесь обоснованно подходить к их выбору, избегая простого перебора;

**Максимальный балл** - 10 (+ бонусы за Kaggle, см. ниже).


Мягкий дедлайн (окончание соревнования на Kaggle): **15 марта 23:59**


In [1]:
%pip install catboost -q


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
from catboost import CatBoostClassifier
import pandas as pd
import os
from tqdm import tqdm
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import scipy


In [2]:
INPUT_DIR = 'data'

train_transaction = pd.read_csv(os.path.join(INPUT_DIR, 'train_transaction.csv'))
train_identity = pd.read_csv(os.path.join(INPUT_DIR, 'train_identity.csv'))
test_transaction = pd.read_csv(os.path.join(INPUT_DIR, 'test_transaction.csv'))
test_identity = pd.read_csv(os.path.join(INPUT_DIR, 'test_identity.csv'))
sample_submission = pd.read_csv(os.path.join(INPUT_DIR, 'sample_submission.csv'))

df_train = train_transaction.merge(train_identity, how='left', on='TransactionID')
df_test = test_transaction.merge(test_identity, how='left', on='TransactionID')

### **Примечания:**

**1. Оценка качества и Submission File**
- Ответом является число от 0 до 1, метрикой качества - AUC-ROC.
- Структура Submission File:
 - для каждого значения *TransactionID* в тестовых данных вы должны предсказать **вероятность** для столбца *isFraud*.
 - в файле у вас должно быть две колонки: `TransactionID` и`isFraud`  **для каждой транзакции в датасете**.

**2. Объем данных**

Поскольку набор данных объемный, могут быть проблемы с переполнением памяти в Collab. Для решения проблемы можете использовать функцию из [этого ноутбука](https://colab.research.google.com/drive/18u75eyFGEoyeWJ_MbsLkcPa6gv2tNI8G#scrollTo=V2L1Nl5CTMMl), разобравшись, что она делает с данными.

# **Задание 2 (3 балла)**

Обучите несколько ML-моделей для решения поставленной задачи.
Оцените их качество двумя способами:

1) на кросс-валидации

2) на лидерборде

Подберите число фолдов на кросс-валидации так, чтобы метрики, которые вы видите, были максимально близки на кросс-валидации и на лидерборде.

По результатам экспериментов постройте таблицу:
* в каждой строке таблицы - результаты одной модели
* по столбцам: качество на кросс-валидации, качество на лидерборде, модель с гиперпараметрами
Полученную таблицу вставьте картинкой прямо в ноутбук после ячеек с кодом. Сделайте текстовые выводы.

## Загрузка данных
В предыдущей части было подготовлено 2 дополнительных пары датасетов - с отобранными признаками и с заполненными пропусками.
Итого имеем 3 набора данных

1. оригинальный с минимальной обработкой
2. с отобранными признаками (с пропусками)
3. с отобранными признаками и заполненными пропусками

In [17]:
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import catboost as cb
import xgboost as xgb
from tqdm import tqdm
import datetime
import joblib


**Вспомогательная функция для экономии памяти**  
Взята из ноутбука, который приводили в пример. 
Моя реализация выше отличается.

In [2]:
def reduce_mem_usage(df):
    NAlist = [] # Keeps track of columns that have missing values filled in.
    for col in tqdm(df.columns):
        if df[col].dtype != object:  # Exclude strings

            # make variables for Int, max and min
            IsInt = False
            col_max_value = df[col].max()
            col_min_value = df[col].min()

            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(df[col]).all():
                NAlist.append(col)
                df[col] = df[col].fillna(col_min_value - 1)
                col_min_value -= 1

            # test if column can be converted to an integer
            col_as_int = df[col].fillna(0).astype(np.int64)
            diff = (df[col] - col_as_int)
            diff = diff.sum()
            if np.abs(diff) < 0.01:
                IsInt = True

            # Make Integer/unsigned Integer datatypes
            if IsInt:
                try:
                    if col_min_value >= 0:
                        if col_max_value < 255:
                            df[col] = df[col].astype(np.uint8)
                        elif col_max_value < 65535:
                            df[col] = df[col].astype(np.uint16)
                        elif col_max_value < 4294967295:
                            df[col] = df[col].astype(np.uint32)
                        else:
                            df[col] = df[col].astype(np.uint64)
                    else:
                        if col_min_value > np.iinfo(np.int8).min and col_max_value < np.iinfo(np.int8).max:
                            df[col] = df[col].astype(np.int8)
                        elif col_min_value > np.iinfo(np.int16).min and col_max_value < np.iinfo(np.int16).max:
                            df[col] = df[col].astype(np.int16)
                        elif col_min_value > np.iinfo(np.int32).min and col_max_value < np.iinfo(np.int32).max:
                            df[col] = df[col].astype(np.int32)
                        elif col_min_value > np.iinfo(np.int64).min and col_max_value < np.iinfo(np.int64).max:
                            df[col] = df[col].astype(np.int64)
                except Exception as e:
                    print(f'Ошибка конвертации {col}: {e}')

            # Make float datatypes 32 bit
            else:
                df[col] = df[col].astype(np.float32)

    return df, NAlist

### Исходные данные

In [3]:
INPUT_DIR = 'data'

train_transaction = pd.read_csv(os.path.join(INPUT_DIR, 'train_transaction.csv'))
train_identity = pd.read_csv(os.path.join(INPUT_DIR, 'train_identity.csv'))
test_transaction = pd.read_csv(os.path.join(INPUT_DIR, 'test_transaction.csv'))
test_identity = pd.read_csv(os.path.join(INPUT_DIR, 'test_identity.csv'))
sample_submission = pd.read_csv(os.path.join(INPUT_DIR, 'sample_submission.csv'))

df_train = train_transaction.merge(train_identity, how='left', on='TransactionID')
del train_transaction, train_identity
df_train, df_train_NAlist = reduce_mem_usage(df_train)

df_test = test_transaction.merge(test_identity, how='left', on='TransactionID')
del test_transaction, test_identity
df_test, df_test_NAlist = reduce_mem_usage(df_test)

df_train.info(), df_test.info()

100%|██████████| 433/433 [00:00<00:00, 596.97it/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 417559 entries, 0 to 417558
Columns: 434 entries, TransactionID to DeviceInfo
dtypes: float32(80), int16(30), int8(202), object(31), uint16(22), uint32(3), uint8(66)
memory usage: 379.1+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172981 entries, 0 to 172980
Columns: 433 entries, TransactionID to DeviceInfo
dtypes: float32(78), int16(40), int8(229), object(31), uint16(24), uint32(3), uint8(28)
memory usage: 157.9+ MB


(None, None)

В данных есть пропуски

In [4]:
print('Missing data in train: {:.5f}%'.format(df_train.isnull().sum().sum() / (df_train.shape[0] * df_train.shape[1]) * 100))
print('Missing data in test: {:.5f}%'.format(df_test.isnull().sum().sum() / (df_test.shape[0] * df_test.shape[1]) * 100))

Missing data in train: 4.47002%
Missing data in test: 4.33051%


Заполним пропуски в столбцах, где значения выражаются числами - `-1`, а где строками - `'unseen_category'`.


In [5]:
for col in df_train.columns.drop('isFraud'):
    if df_train[col].dtype == 'O':
        df_train[col] = df_train[col].fillna('unseen_category')
        df_test[col] = df_test[col].fillna('unseen_category')
    else:
        df_train[col] = df_train[col].fillna(-1)
        df_test[col] = df_test[col].fillna(-1)

print('Missing data in train: {:.5f}%'.format(df_train.isnull().sum().sum() / (df_train.shape[0] * df_train.shape[1]) * 100))
print('Missing data in test: {:.5f}%'.format(df_test.isnull().sum().sum() / (df_test.shape[0] * df_test.shape[1]) * 100))

Missing data in train: 0.00000%
Missing data in test: 0.00000%


Закодируем категориальные признаки с помощью [`LabelEncoder`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html) и сконвертируем их в [`category`](https://pandas.pydata.org/pandas-docs/version/0.23.4/categorical.html).

In [6]:
for col in tqdm(df_train.columns.drop('isFraud')):
    if df_train[col].dtype == 'O':
        le = LabelEncoder()
        le.fit(list(df_train[col]) + list(df_test[col]))
        df_train[col] = le.transform(df_train[col])
        df_test[col] = le.transform(df_test[col])

        df_train[col] = df_train[col].astype('category')
        df_test[col] = df_test[col].astype('category')

df_train.info(), df_test.info()

100%|██████████| 433/433 [00:10<00:00, 39.96it/s] 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 417559 entries, 0 to 417558
Columns: 434 entries, TransactionID to DeviceInfo
dtypes: category(31), float32(80), int16(30), int8(202), uint16(22), uint32(3), uint8(66)
memory usage: 293.5 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172981 entries, 0 to 172980
Columns: 433 entries, TransactionID to DeviceInfo
dtypes: category(31), float32(78), int16(40), int8(229), uint16(24), uint32(3), uint8(28)
memory usage: 122.5 MB


(None, None)

### Данные с отобранными признаками

In [7]:
X_train = pd.read_csv('./data/X_train.csv')
X_test = pd.read_csv('./data/X_test.csv')

X_train.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
X_test.drop(columns=['Unnamed: 0'], axis=1, inplace=True)

### Данные с заполненными пропусками

In [8]:
X_train_processed = pd.read_csv('./data/X_train_processed.csv')
X_test_processed = pd.read_csv('./data/X_test_processed.csv')

X_train_processed.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
X_test_processed.drop(columns=['Unnamed: 0'], axis=1, inplace=True)

### Целевая переменная

In [9]:
y_train=pd.read_csv('./data/y_train.csv')
y_train.drop(columns=['Unnamed: 0'], axis=1, inplace=True)

### Выравнивание типов данных

In [10]:
# выравнивание типов данных

X_train_cat = X_train.copy()
for col in X_train_cat.columns:
    if X_train_cat[col].dtype == 'O':
        X_train_cat[col] = X_train_cat[col].fillna('unseen_category')
        X_train_cat[col] = X_train_cat[col].astype('category')
X_train_cat.info()

X_train_processed_cat = X_train_processed.copy()
for col in X_train_processed.columns:
    if X_train_processed_cat[col].dtype == 'O':
        X_train_processed_cat[col] = X_train_processed_cat[col].fillna('unseen_category')
        X_train_processed_cat[col] = X_train_processed_cat[col].astype('category')
X_train_processed_cat.info()

X_test_cat = X_test.copy()
for col in X_test_cat.columns:
    if X_test_cat[col].dtype == 'O':
        X_test_cat[col] = X_test_cat[col].fillna('unseen_category')
        X_test_cat[col] = X_test_cat[col].astype('category')
X_test_cat.info()

X_test_processed_cat = X_test_processed.copy()
for col in X_train_processed.columns:
    if X_test_processed_cat[col].dtype == 'O':
        X_test_processed_cat[col] = X_test_processed_cat[col].fillna('unseen_category')
        X_test_processed_cat[col] = X_test_processed_cat[col].astype('category')
X_test_processed_cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 417559 entries, 0 to 417558
Columns: 334 entries, TransactionAmt to DeviceInfo
dtypes: category(31), float64(257), int64(46)
memory usage: 978.5 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 417559 entries, 0 to 417558
Columns: 498 entries, TransactionAmt to M3_missing
dtypes: category(29), float64(128), int64(341)
memory usage: 1.5 GB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172981 entries, 0 to 172980
Columns: 334 entries, TransactionAmt to DeviceInfo
dtypes: category(31), float64(271), int64(32)
memory usage: 405.4 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172981 entries, 0 to 172980
Columns: 498 entries, TransactionAmt to M3_missing
dtypes: category(29), float64(128), int64(341)
memory usage: 624.1 MB


## Подготовка кросс-валидации

Я уже провел календарный анализ и показал, что признак `TransactionDT` задан в секундах, а обучающая выборка - это данные за 4 месяца с `07.12.2018` до `06.04.2019`.

Тогда логично в качестве фолдов для кроссвалидации взять последовательные подвыборки, соответствующие одному месяцу. Это позволит сохранить распределение признаков внутри фолдов.

Зададим фолды при помощи индексов, тогда их можно будет применять ко всем трем наборам данных.

In [11]:
month_length = 3600 * 24 * 30

fold0_idx = df_train[df_train['TransactionDT'] < df_train['TransactionDT'].min() + month_length].index
fold1_idx = df_train[(df_train['TransactionDT'].min() + month_length <= df_train['TransactionDT']) & (df_train['TransactionDT'] < df_train['TransactionDT'].min() + 2 * month_length)].index
fold2_idx = df_train[(df_train['TransactionDT'].min() + 2 * month_length <= df_train['TransactionDT']) & (df_train['TransactionDT'] < df_train['TransactionDT'].min() + 3 * month_length)].index
fold3_idx = df_train[df_train['TransactionDT'].min() + 3 * month_length <= df_train['TransactionDT']].index
folds_idx = [fold0_idx, fold1_idx, fold2_idx, fold3_idx]

print('Validation set 0 length:', len(fold0_idx))
print('Validation set 1 length:', len(fold1_idx))
print('Validation set 2 length:', len(fold2_idx))
print('Validation set 3 length:', len(fold3_idx))


Validation set 0 length: 134339
Validation set 1 length: 89399
Validation set 2 length: 92189
Validation set 3 length: 101632


В данных есть признак-идентификатор объекта - `'TransactionID'`. Заметим, что его значения в обучающей и тестовых выборках не пересекаются:

In [12]:
set(df_train['TransactionID']).intersection(set(df_test['TransactionID']))

set()

Также не пересекаются значения признака, отвечающего за момент времени - `'TransactionDT'`:

In [13]:
set(df_train['TransactionDT']).intersection(set(df_test['TransactionDT']))

set()

Поэтому удалим эти признаки, чтобы модель их не учитывала.  
И удалим `isFraud`  из `df_train`

In [14]:
df_train.drop(['TransactionID', 'TransactionDT', 'isFraud'], axis=1, inplace=True)
df_test.drop(['TransactionID', 'TransactionDT'], axis=1, inplace=True)
df_train.shape, df_test.shape

((417559, 431), (172981, 431))

## Обучение моделей

In [18]:
def train_and_predict(wrapper_class, params, df_train_num, df_train_cat_encoded, df_train_cat_processed,
                     df_test_num, df_test_cat_encoded, df_test_cat_processed,
                     y_train, folds_idx, sample_submission,
                     save_data=True, save_models=True, submit_to_kaggle=True):
    """
    Обучает три модели на разных наборах признаков и делает предсказания
    
    Args:
        wrapper_class: Класс-обертка для модели (XGBoostWrapper, LightGBMWrapper и т.д.)
        params: Параметры модели
        df_train_num: Датафрейм с числовыми признаками для обучения
        df_train_cat_encoded: Датафрейм с закодированными категориальными признаками для обучения
        df_train_cat_processed: Датафрейм с обработанными категориальными признаками для обучения
        df_test_num: Тестовый датафрейм с числовыми признаками
        df_test_cat_encoded: Тестовый датафрейм с закодированными категориальными признаками
        df_test_cat_processed: Тестовый датафрейм с обработанными категориальными признаками
        y_train: Целевая переменная
        folds_idx: Индексы фолдов для кросс-валидации
        sample_submission: Датафрейм с примером сабмита
        save_data: Сохранять ли предсказания в файлы
        save_models: Сохранять ли обученные модели
        submit_to_kaggle: Отправлять ли результаты на Kaggle
    """
    
    print('\nОбучаем модель 1 (числовые + закодированные категориальные признаки)...')
    model1 = wrapper_class(params=params)
    score1 = model1.fit(df_train_num, y_train, folds_idx)
    print(f'CV AUC-ROC: {score1:.5f}')

    print('\nОбучаем модель 2 (числовые + закодированные категориальные признаки)...')
    model2 = wrapper_class(params=params)
    score2 = model2.fit(df_train_cat_encoded, y_train, folds_idx)
    print(f'CV AUC-ROC: {score2:.5f}')

    print('\nОбучаем модель 3 (числовые + обработанные категориальные признаки)...')
    model3 = wrapper_class(params=params)
    score3 = model3.fit(df_train_cat_processed, y_train, folds_idx)
    print(f'CV AUC-ROC: {score3:.5f}')

    # Делаем предсказания
    print('\nДелаем предсказания...')
    y_pred1 = model1.predict(df_test_num)
    y_pred2 = model2.predict(df_test_cat_encoded)
    y_pred3 = model3.predict(df_test_cat_processed)
    y_pred4 = (y_pred1 + y_pred2 + y_pred3) / 3

    print(f'Модель 1 (числовые признаки): {y_pred1.shape}')
    print(f'Модель 2 (закодированные категориальные признаки): {y_pred2.shape}')
    print(f'Модель 3 (обработанные категориальные признаки): {y_pred3.shape}')
    print(f'Модель 4 (усреднение 1, 2 и 3): {y_pred4.shape}')

    if save_data:
        # Создаем сабмиты
        sub1 = pd.DataFrame({'TransactionID': sample_submission['TransactionID'], 'isFraud': y_pred1})
        sub2 = pd.DataFrame({'TransactionID': sample_submission['TransactionID'], 'isFraud': y_pred2})
        sub3 = pd.DataFrame({'TransactionID': sample_submission['TransactionID'], 'isFraud': y_pred3})
        sub4 = pd.DataFrame({'TransactionID': sample_submission['TransactionID'], 'isFraud': y_pred4})

        # Сохраняем сабмиты
        sub1.to_csv('data/submission_model1.csv', index=False)
        sub2.to_csv('data/submission_model2.csv', index=False)
        sub3.to_csv('data/submission_model3.csv', index=False)
        sub4.to_csv('data/submission_model4.csv', index=False)

    if submit_to_kaggle:
        # Отправляем результаты
        !kaggle competitions submit -c fraud-detection-24 -f data/submission_model1.csv -m "Task2: model1"
        !kaggle competitions submit -c fraud-detection-24 -f data/submission_model2.csv -m "Task2: model2"
        !kaggle competitions submit -c fraud-detection-24 -f data/submission_model3.csv -m "Task2: model3"
        !kaggle competitions submit -c fraud-detection-24 -f data/submission_model4.csv -m "Task2: model4"

    if save_models:
        # Сохраняем модели
        joblib.dump(model1, 'models/model1.joblib')
        joblib.dump(model2, 'models/model2.joblib')
        joblib.dump(model3, 'models/model3.joblib')
        
    return model1, model2, model3, (y_pred1, y_pred2, y_pred3, y_pred4)


### LightGBM

#### Fit

In [19]:
from sklearn.base import BaseEstimator


class LightGBMWrapper(BaseEstimator):
    def __init__(self, params):
        self.params = params
        self.models = []
        self.scores = []
        self.models = []
        
    def fit(self, X: pd.DataFrame, y: pd.Series, folds_idx: list[np.ndarray]):
        for i in range(len(folds_idx)):
            _X_train = X.drop(folds_idx[i], axis=0)
            _y_train = y.drop(folds_idx[i], axis=0)
            _X_val = X.iloc[folds_idx[i]]
            _y_val = y.iloc[folds_idx[i]]
            
            lgb_train = lgb.Dataset(_X_train, _y_train)
            lgb_eval = lgb.Dataset(_X_val, _y_val, reference=lgb_train)
            
            _model = lgb.train(self.params, lgb_train, valid_sets=lgb_eval)
            self.models.append(_model)
            
            _y_pred = _model.predict(_X_val)
            score_fold = roc_auc_score(_y_val, _y_pred)
            self.scores.append(score_fold)
            
        return np.mean(self.scores)
    
    def predict(self, X):
        if not self.models:
            raise ValueError("Модель не обучена. Сначала выполните fit()")
            
        predictions = []
        for model in self.models:
            predictions.append(model.predict(X))
        return np.mean(predictions, axis=0)


In [20]:
params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'metric': 'auc',
    'n_jobs': -1,
    'n_estimators': 2000,
    'seed': 42,
    'early_stopping_rounds': 200,
    # 'device': "gpu",
}

In [21]:
lgb1, lgb2, lgb3, (y_pred1, y_pred2, y_pred3, y_pred4) = train_and_predict(LightGBMWrapper, params, df_train, X_train_cat, X_train_processed_cat,
                  df_test, X_test_cat, X_test_processed_cat,
                  y_train, folds_idx, sample_submission, save_data=False, save_models=False, submit_to_kaggle=False)


Обучаем модель 1 (числовые + закодированные категориальные признаки)...


/Users/yukonoff/code/hse_ai_24/hse-ai24-ml/.venv/lib/python3.12/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Number of positive: 11320, number of negative: 271900
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.158217 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 31545
[LightGBM] [Info] Number of data points in the train set: 283220, number of used features: 429
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.039969 -> initscore=-3.178863
[LightGBM] [Info] Start training from score -3.178863
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[575]	valid_0's auc: 0.900345


/Users/yukonoff/code/hse_ai_24/hse-ai24-ml/.venv/lib/python3.12/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Number of positive: 11144, number of negative: 317016
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.206276 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 34468
[LightGBM] [Info] Number of data points in the train set: 328160, number of used features: 429
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.033959 -> initscore=-3.348051
[LightGBM] [Info] Start training from score -3.348051
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[591]	valid_0's auc: 0.9241


...

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Number of positive: 11144, number of negative: 317016
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.233838 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 36660
[LightGBM] [Info] Number of data points in the train set: 328160, number of used features: 495
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.033959 -> initscore=-3.348051
[LightGBM] [Info] Start training from score -3.348051
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[520]	valid_0's auc: 0.925485


/Users/yukonoff/code/hse_ai_24/hse-ai24-ml/.venv/lib/python3.12/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Number of positive: 10997, number of negative: 314373
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.211044 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 36500
[LightGBM] [Info] Number of data points in the train set: 325370, number of used features: 494
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.033798 -> initscore=-3.352958
[LightGBM] [Info] Start training from score -3.352958
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[466]	valid_0's auc: 0.923908


/Users/yukonoff/code/hse_ai_24/hse-ai24-ml/.venv/lib/python3.12/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Number of positive: 10702, number of negative: 305225
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.194183 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 36729
[LightGBM] [Info] Number of data points in the train set: 315927, number of used features: 495
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.033875 -> initscore=-3.350619
[LightGBM] [Info] Start training from score -3.350619
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[209]	valid_0's auc: 0.903265
CV AUC-ROC: 0.91343

Делаем предсказания...
Модель 1 (числовые признаки): (17298

In [ ]:
# lgb1 = LightGBMWrapper(params=params)
# score1 = lgb1.fit(df_train, y_train, folds_idx)
# print('CV AUC-ROC: {:.5f}'.format(np.mean(score1)))

# lgb2 = LightGBMWrapper(params=params)
# score2 = lgb2.fit(X_train_cat, y_train, folds_idx)
# print('CV AUC-ROC: {:.5f}'.format(np.mean(score2)))

# lgb3 = LightGBMWrapper(params=params)
# score3 = lgb3.fit(X_train_processed_cat, y_train, folds_idx)
# print('CV AUC-ROC: {:.5f}'.format(np.mean(score3)))


#### Predict

In [25]:
# # Предсказания для всех трех моделей
# y_pred1 = lgb1.predict(df_test)
# y_pred2 = lgb2.predict(X_test_cat)
# y_pred3 = lgb3.predict(X_test_processed_cat)
# # Усредняем предсказания всех трех моделей
# y_pred4 = (y_pred1 + y_pred2 + y_pred3) / 3


# print('Предсказания сделаны для всех трех моделей:')
# print(f'Модель 1 (базовая): {y_pred1.shape}')
# print(f'Модель 2 (категориальные признаки): {y_pred2.shape}') 
# print(f'Модель 3 (обработанные категориальные признаки): {y_pred3.shape}')
# print(f'Модель 4 (усреднение 1, 2 и 3): {y_pred4.shape}')



Предсказания сделаны для всех трех моделей:
Модель 1 (базовая): (172981,)
Модель 2 (категориальные признаки): (172981,)
Модель 3 (обработанные категориальные признаки): (172981,)
Модель 4 (усреднение 1, 2 и 3): (172981,)


In [22]:
sub1 = pd.DataFrame({'TransactionID': sample_submission['TransactionID'], 'isFraud': y_pred1})
sub2 = pd.DataFrame({'TransactionID': sample_submission['TransactionID'], 'isFraud': y_pred2})
sub3 = pd.DataFrame({'TransactionID': sample_submission['TransactionID'], 'isFraud': y_pred3})
sub4 = pd.DataFrame({'TransactionID': sample_submission['TransactionID'], 'isFraud': y_pred4})

sub4.head()

,TransactionID,isFraud
0,3404559,0.006539
1,3404560,0.060885
2,3404561,0.013823
3,3404562,0.006402
4,3404563,0.500748


In [23]:
from datetime import datetime
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
sub1.to_csv(f'data/submission_lgbm1_{timestamp}.csv', index=False) # Score: 0.91599
sub2.to_csv(f'data/submission_lgbm2_{timestamp}.csv', index=False) # Score: 0.91674
sub3.to_csv(f'data/submission_lgbm3_{timestamp}.csv', index=False) # Score: 0.91699
sub4.to_csv(f'data/submission_lgbm4_{timestamp}.csv', index=False) # Score: 0.92049

In [24]:
# # Отправка файла на соревнование
!kaggle competitions submit -c fraud-detection-24 -f data/submission_lgbm1_{timestamp}.csv -m "Task2: lgbm1"
!kaggle competitions submit -c fraud-detection-24 -f data/submission_lgbm2_{timestamp}.csv -m "Task2: lgbm2"
!kaggle competitions submit -c fraud-detection-24 -f data/submission_lgbm3_{timestamp}.csv -m "Task2: lgbm3"
!kaggle competitions submit -c fraud-detection-24 -f data/submission_lgbm4_{timestamp}.csv -m "Task2: lgbm4"

100%|██████████████████████████████████████| 4.81M/4.81M [00:01<00:00, 2.94MB/s]
100%|██████████████████████████████████████| 4.81M/4.81M [00:01<00:00, 2.98MB/s]
100%|██████████████████████████████████████| 4.80M/4.80M [00:01<00:00, 3.11MB/s]
100%|██████████████████████████████████████| 4.80M/4.80M [00:01<00:00, 3.11MB/s]
Successfully submitted to Fraud Detection Competition

In [55]:
# Сохраняем все модели.
# В LightGBM есть встроенная функция сохранения модели, но у нас самодельная кросс-валидация и 4 модели внутри.
# Поэтому сохраняем вручную.
import joblib

joblib.dump(lgb1, f'models/model_lgb1_{timestamp}.joblib')
joblib.dump(lgb2, f'models/model_lgb2_{timestamp}.joblib')
joblib.dump(lgb3, f'models/model_lgb3_{timestamp}.joblib')

['models/model_lgb3.joblib']

#### Результаты и выводы

**Подход к построению моделей**
1. Были обучены 3 базовые модели LightGBM на разных наборах данных:
   - `lgb1` - на исходных данных с базовой предобработкой
   - `lgb2` - на данных с отобранными признаками
   - `lgb3` - на данных с заполненными пропусками и дополнительной обработкой
2. Использована кастомная реализация кросс-валидации:
   - 4 фолда, разделенные по временным периодам (по месяцам)
   - Это позволило сохранить временную структуру данных
   - Каждая модель обучалась отдельно на каждом фолде
  
Параметры всех моделей
```python
params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'metric': 'auc',
    'n_jobs': -1,
    'n_estimators': 2000,
    'seed': 13,
    'early_stopping_rounds': 200
}
```
**Результаты**
| Модель                              | AUC-ROC |  Kaggle |
|-------------------------------------|---------|---------|
| `lbg1` (базовая)                    | 0.91476 | 0.91599 |
| `lgb2` (с отобранными признаками)   | 0.91271 | 0.91674 |
| `lgb3` (с обработанными признаками) | 0.91269 | 0.91699 |
| Ансамбль (среднее трёх моделей)     |       - | 0.92049 |

AUC-ROC ансамбля не указан, т.к. внутри моделей скрыта кастомная кроссвалидация и эти значения AUC-ROC - это уже усредненные AUC-ROC на фолдах.

**Выводы**

1. **Стабильность**:  
        Все три модели показали близкие результаты как на кросс-валидации (CV), так и на тестовой выборке (Kaggle), что говорит о стабильности подхода.
2. **Эффективность ансамблирования**:  
        Усреднение предсказаний дало значительное улучшение результата (0.92049), что превосходит каждую индивидуальную модель на 0.4%-0.6%
3. **Соответствие CV и лидерборда**:  
        Результаты на CV хорошо коррелируют с результатами на Kaggle, что подтверждает корректность выбранной схемы валидации.
4. **Влияние предобработки**:  
        Интересно что более сложная предобработка данных (модели 2 и 3) не дала существенного преимущества на CV, но показала лучшие результаты на тестовой выборке. 

> Лучшим решением оказался ансамбль всех трёх моделей, что подчеркивает важность разнообразия подходов к обработке данных и построению моделей.

### Catboost

#### Fit

In [22]:
from sklearn.base import BaseEstimator
import catboost as cb


class CatboostWrapper(BaseEstimator):
    def __init__(self, params):
        self.params = params
        self.models = []
        self.scores = []
        
    def fit(self, X: pd.DataFrame, y: pd.Series, folds_idx: list[np.ndarray]):
        cat_features = X.select_dtypes(include=['category']).columns.tolist()
        for i in range(len(folds_idx)):
            _X_train = X.drop(folds_idx[i], axis=0)
            _y_train = y.drop(folds_idx[i], axis=0)
            _X_val = X.iloc[folds_idx[i]]
            _y_val = y.iloc[folds_idx[i]]
            
            cb_train = cb.Pool(_X_train, _y_train, cat_features=cat_features)
            cb_eval = cb.Pool(_X_val, _y_val, cat_features=cat_features)
            
            model = cb.CatBoostClassifier(**self.params)
            model.fit(cb_train, eval_set=cb_eval)
            self.models.append(model)
            # model.save_model(fname=f'models/cb_fold_{i}_{timestamp}.cbm', format="cbm")

            # _model = lgb.train(self.params, lgb_train, valid_sets=lgb_eval)
            # self.models.append(_model)
            
            _y_pred = model.predict_proba(_X_val)[:, 1]
            score_fold = roc_auc_score(_y_val, _y_pred)
            self.scores.append(score_fold)
            
        return np.mean(self.scores)
    
    def predict(self, X):
        if not self.models:
            raise ValueError("Модель не обучена. Сначала выполните fit()")
            
        predictions = []
        for model in self.models:
            predictions.append(model.predict(X))
        return np.mean(predictions, axis=0)


In [23]:
params = {
    'objective': 'Logloss',
    'eval_metric': 'AUC',
    'learning_rate': 0.2,
    'iterations': 200,
    'random_seed': 42,
    'early_stopping_rounds': 200,
    'verbose': 200,
    # 'task_type': "GPU",
}


In [ ]:
cb1 = CatboostWrapper(params=params)
score1 = cb1.fit(df_train, y_train, folds_idx)
print('CV AUC-ROC: {:.5f}'.format(np.mean(score1)))

cb2 = CatboostWrapper(params=params)
score2 = cb2.fit(X_train_cat, y_train, folds_idx)
print('CV AUC-ROC: {:.5f}'.format(np.mean(score2)))

cb3 = CatboostWrapper(params=params)
score3 = cb3.fit(X_train_processed_cat, y_train, folds_idx)
print('CV AUC-ROC: {:.5f}'.format(np.mean(score3)))

#### Predict

In [ ]:
# Предсказания для всех трех моделей
y_pred1 = cb1.predict(df_test)
y_pred2 = cb2.predict(X_test_cat)
y_pred3 = cb3.predict(X_test_processed_cat)
# Усредняем предсказания всех трех моделей
y_pred4 = (y_pred1 + y_pred2 + y_pred3) / 3


print('Предсказания сделаны для всех трех моделей:')
print(f'Модель 1 (базовая): {y_pred1.shape}')
print(f'Модель 2 (категориальные признаки): {y_pred2.shape}') 
print(f'Модель 3 (обработанные категориальные признаки): {y_pred3.shape}')
print(f'Модель 4 (усреднение 1, 2 и 3): {y_pred4.shape}')

In [ ]:
sub1 = pd.DataFrame({'TransactionID': sample_submission['TransactionID'], 'isFraud': y_pred1})
sub2 = pd.DataFrame({'TransactionID': sample_submission['TransactionID'], 'isFraud': y_pred2})
sub3 = pd.DataFrame({'TransactionID': sample_submission['TransactionID'], 'isFraud': y_pred3})
sub4 = pd.DataFrame({'TransactionID': sample_submission['TransactionID'], 'isFraud': y_pred4})

sub4.head()

In [ ]:
sub1.to_csv('data/submission_cb1.csv', index=False) # Score: (0.91599)
sub2.to_csv('data/submission_cb2.csv', index=False) # Score: (0.91674)
sub3.to_csv('data/submission_cb3.csv', index=False) # Score: (0.91699)
sub4.to_csv('data/submission_cb4.csv', index=False) # Score: (0.92049)

In [1]:
# # Отправка файла на соревнование
!kaggle competitions submit -c fraud-detection-24 -f data/submission_cb1.csv -m "Task2: cb1"
!kaggle competitions submit -c fraud-detection-24 -f data/submission_cb2.csv -m "Task2: cb2"
!kaggle competitions submit -c fraud-detection-24 -f data/submission_cb3.csv -m "Task2: cb3"
!kaggle competitions submit -c fraud-detection-24 -f data/submission_cb4.csv -m "Task2: cb4"

100%|██████████████████████████████████████| 1.98M/1.98M [00:01<00:00, 1.36MB/s]
100%|██████████████████████████████████████| 1.98M/1.98M [00:01<00:00, 1.28MB/s]
100%|██████████████████████████████████████| 1.98M/1.98M [00:01<00:00, 1.52MB/s]
100%|██████████████████████████████████████| 2.01M/2.01M [00:01<00:00, 1.40MB/s]
Successfully submitted to Fraud Detection Competition

In [ ]:
# Сохраняем все модели.
# В LightGBM есть встроенная функция сохранения модели, но у нас самодельная кросс-валидация и 4 модели внутри.
# Поэтому сохраняем вручную.
import joblib

joblib.dump(cb1, 'models/model_cb1.joblib')
joblib.dump(cb2, 'models/model_cb2.joblib')
joblib.dump(cb3, 'models/model_cb3.joblib')

### XGBoost

> Если запускается в разнобой, то предварительно выполнить ячейки
> - Загрузка данных
> - Подготовка кросс-валидации

In [26]:
# Реализуем аналогичный подход с XGBoost

from xgboost import XGBClassifier

class XGBoostWrapper:
    def __init__(self, params=None):
        self.models = []
        self.params = params if params else {
            'n_estimators': 1000,
            'learning_rate': 0.01,
            'max_depth': 5,
            'subsample': 0.8,
            'colsample_bytree': 0.8,
            'objective': 'binary:logistic',
            'eval_metric': 'auc',
            'random_state': 42,
            'n_jobs': -1
        }
        
    def fit(self, X, y, folds_idx):
        scores = []
        for i, fold_idx in enumerate(folds_idx):
            X_train = X.drop(fold_idx, axis=0)
            y_train = y.drop(fold_idx, axis=0)
            X_valid = X.iloc[fold_idx]
            y_valid = y.iloc[fold_idx]
             
            model = XGBClassifier(**self.params)
            # В новых версиях XGBoost early_stopping_rounds передается через fit_params
            model.fit(
                X_train, y_train,
                eval_set=[(X_valid, y_valid)],
                verbose=100
            )
            
            self.models.append(model)
            score = roc_auc_score(y_valid, model.predict_proba(X_valid)[:, 1])
            scores.append(score)
            print(f'Fold {i + 1} ROC AUC: {score}')
            
        print(f'Mean ROC AUC: {np.mean(scores)}')
        return np.mean(scores)
    
    def predict(self, X):
        preds = []
        for model in self.models:
            pred = model.predict_proba(X)[:, 1]
            preds.append(pred)
        return np.mean(preds, axis=0)

In [29]:
params = {
    'n_estimators': 1000,
    'learning_rate': 0.01,
    'max_depth': 7,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'random_state': 42,
    'n_jobs': -1,
    'early_stopping_rounds': 200,  # Добавляем early_stopping_rounds в параметры модели
    'enable_categorical': True
}

In [32]:
# Обучаем три модели с разными наборами признаков
xgb1, xgb2, xgb3, (y_pred1, y_pred2, y_pred3, y_pred4) = train_and_predict(XGBoostWrapper, params, df_train, X_train_cat, X_train_processed_cat,
                  df_test, X_test_cat, X_test_processed_cat,
                  y_train, folds_idx, sample_submission, save_data=False, save_models=False, submit_to_kaggle=False)


Обучаем модель 1 (числовые + закодированные категориальные признаки)...
[0]	validation_0-auc:0.78011
[100]	validation_0-auc:0.83986
[200]	validation_0-auc:0.85323
[300]	validation_0-auc:0.86210
[400]	validation_0-auc:0.86971
[500]	validation_0-auc:0.87576
[600]	validation_0-auc:0.88150
[700]	validation_0-auc:0.88588
[800]	validation_0-auc:0.88915
[900]	validation_0-auc:0.89191
[999]	validation_0-auc:0.89456
Fold 1 ROC AUC: 0.8945600108926582
[0]	validation_0-auc:0.79360
[100]	validation_0-auc:0.86195
[200]	validation_0-auc:0.87727
[300]	validation_0-auc:0.88960
[400]	validation_0-auc:0.89632
[500]	validation_0-auc:0.90056
[600]	validation_0-auc:0.90465
[700]	validation_0-auc:0.90848
[800]	validation_0-auc:0.91108
[900]	validation_0-auc:0.91335
[999]	validation_0-auc:0.91554
Fold 2 ROC AUC: 0.9155386642071525
[0]	validation_0-auc:0.80409
[100]	validation_0-auc:0.88325
[200]	validation_0-auc:0.89480
[300]	validation_0-auc:0.90438
[400]	validation_0-auc:0.90914
[500]	validation_0-auc:0.9

In [36]:
sub1 = pd.DataFrame({'TransactionID': sample_submission['TransactionID'], 'isFraud': y_pred1})
sub2 = pd.DataFrame({'TransactionID': sample_submission['TransactionID'], 'isFraud': y_pred2})
sub3 = pd.DataFrame({'TransactionID': sample_submission['TransactionID'], 'isFraud': y_pred3})
sub4 = pd.DataFrame({'TransactionID': sample_submission['TransactionID'], 'isFraud': y_pred4})

sub4.head()

,TransactionID,isFraud
0,3404559,0.033731
1,3404560,0.330279
2,3404561,0.329900
3,3404562,0.038083
4,3404563,0.358853


In [37]:
from datetime import datetime
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
sub1.to_csv(f'data/submission_xgb1_{timestamp}.csv', index=False) # Score: 0.91599
sub2.to_csv(f'data/submission_xgb2_{timestamp}.csv', index=False) # Score: 0.91674
sub3.to_csv(f'data/submission_xgb3_{timestamp}.csv', index=False) # Score: 0.91699
sub4.to_csv(f'data/submission_xgb4_{timestamp}.csv', index=False) # Score: 0.92049

In [38]:
# # Отправка файла на соревнование
!kaggle competitions submit -c fraud-detection-24 -f data/submission_xgb1_{timestamp}.csv -m "Task2: xgb1"
!kaggle competitions submit -c fraud-detection-24 -f data/submission_xgb2_{timestamp}.csv -m "Task2: xgb2"
!kaggle competitions submit -c fraud-detection-24 -f data/submission_xgb3_{timestamp}.csv -m "Task2: xgb3"
!kaggle competitions submit -c fraud-detection-24 -f data/submission_xgb4_{timestamp}.csv -m "Task2: xgb4"

100%|██████████████████████████████████████| 3.21M/3.21M [00:01<00:00, 2.10MB/s]
100%|██████████████████████████████████████| 3.22M/3.22M [00:01<00:00, 2.20MB/s]
100%|██████████████████████████████████████| 3.27M/3.27M [00:01<00:00, 2.19MB/s]
100%|██████████████████████████████████████| 3.23M/3.23M [00:01<00:00, 2.18MB/s]
Successfully submitted to Fraud Detection Competition

# **Задание 3 (2 балла)**

Попробуйте подойти к задаче как к поиску аномалий.

1) Поищите аномалии (фрод) различными рассмотренными в курсе методами и сделайте прогноз на тестовых данных.

Результатом также будет таблица:
* по строкам - методы поиска аномалий
* по столбцам - качество вашего решения на leaderboard

2) Попробуйте встроить поиск аномалий и их удаление в ML-пайплайн: найдите аномалии и что-нибудь с ними сделайте до обучения моделей (можно удалить их, а можно использовать в качестве дополнительных признаков - попробуйте разные стратегии). Результат проверьте на кросс-валидации и на лидерборде, сделайте выводы.

In [ ]:
# ваша работа с аномалиями здесь

# **Задание 4 (1 балл)**

Сделайте кластеризацию различными способами. Результаты кластеризации используйте для улучшения ML-решений:

1) Номера кластеров закодируйте (OHE или target-encoding) и добавьте как новые признаки

2) При использовании DBSCAN / HDBSCAN предсказанный шум можно трактовать как найденную аномалию и также добавить ее как новый признак

Проведите различные эксперименты. Проверьте как эти подходы влияют на качество прогнозов по кросс-валидации и на лидерборде, сделайте выводы.

In [ ]:
# ваши эксперименты с кластеризацией здесь

## **Задание 5 (1 балл)**

Примените какой-нибудь (один любой) AutoML фреймворк для решения поставленной задачи.

Отправьте AutoML-прогноз на kaggle и посмотрите на качество модели. Сделайте текстовые выводы.

In [ ]:
# ваш AutoML здесь

# **Задание 6 (1 балл)**

Весь курс мы работали в Google Colab. Но всегда должны быть запасные варианты, где Вы будете обучать модели.

Среди вариантов есть:
* ваша локальная машина
* kaggle notebooks
* yandex cloud
и другие.

Кроме привычного Google Colab выберите из списка выше один любой альтернативный вариант и проведите эксперимент:

* Прогоните ваш лучший по качеству по результатам заданий 2-4 ML-пайплайн заново в Google Colab и с помощью библиотек (например, при помощи библиотеки time) замерьте время обучения и отдельно время инференса на тестовых данных

* Прогоните этот пайплайн на выбранном альтернативном сервисе/локальной машине и также замерьте время обучения и инференса.

Текстом напишите выводы: опишите, какое альтернативное место для обучения моделей Вы использовали? Прикрепите прямо в ноутбук скриншот с экраном кода в альтернативном сервисе/на локальной машине. Также в виде таблицы приведите сравнение времени обучения и инференса в колабе и в альтернативном месте. Сделайте выводы.

In [ ]:
# ваши эксперименты здесь

# **Бонус: за Kaggle и стремление к хорошим скорам (2 балла)**

В этом домашнем задании Ваша цель - не просто выполнить шаги выше, но и построить максимально хорошую по качеству модель.

**К 10 вы можете получить до двух дополнительных баллов:**

* За попадание в топ-20% на private leaderboard — +1 дополнительный балл к оценке
* За попадание в топ-5 мест на private leaderboard — + еще один дополнительный балл к оценке (то есть суммарно 2 дополнительных балла)

**ВАЖНО!!!**

Эти баллы ставятся до мягкого дедлайна по соревнованию. После мягкого дедлайна лидерборд не обновляется, и дополнительные баллы не ставятся.

Успехов!

In [ ]:
# не забудьте прикрепить скриншоты лидерборда, пожалуйста